In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from IPython.display import display, Markdown

os.makedirs('../graphs', exist_ok=True)

# Load data
dfs = []
for year in range(2021, 2026):
    dfs.append(pd.read_csv(f'../data/processed/poles_{year}.csv'))

df_all = pd.concat(dfs, ignore_index=True)

print(f"Loaded {df_all.shape[0]} races (2021–2025)")

# Type cleanup
df_all['PoleFinishPosition'] = pd.to_numeric(df_all['PoleFinishPosition'], errors='coerce')
df_all['Lap1Position'] = pd.to_numeric(df_all['Lap1Position'], errors='coerce')
df_all['WonFromPole'] = df_all['WonFromPole'].astype(bool)

# Feature engineering
df_all['Lap1Change'] = df_all['PoleFinishPosition'] - df_all['Lap1Position']

df_all['PoleStartVsLap1'] = df_all['Lap1Change'].apply(
    lambda x: 'Gained' if x > 0 else ('Lost' if x < 0 else 'Maintained')
)

df_all['PoleToFinishChange'] = 1 - df_all['PoleFinishPosition']

# Filter drivers with ≥3 poles
driver_counts = df_all.groupby('PoleDriver').size()
drivers_3plus = driver_counts[driver_counts >= 3].index
df_filtered = df_all[df_all['PoleDriver'].isin(drivers_3plus)]

print(f"Drivers with ≥3 poles: {list(drivers_3plus)}")

# Pole conversion
pole_counts = df_filtered.groupby('PoleDriver').size().reset_index(name='Poles')
pole_conversion = (
    df_filtered.groupby('PoleDriver')['WonFromPole']
    .mean()
    .reset_index()
    .merge(pole_counts, on='PoleDriver')
)

pole_conversion['ConversionRate'] = (pole_conversion['WonFromPole'] * 100).round(0).astype(int)
pole_conversion = pole_conversion.sort_values('ConversionRate', ascending=False)

print("\n=== Pole Conversion Summary ===")
print(pole_conversion[['PoleDriver', 'Poles', 'ConversionRate']])

# Graph 1: Scatter plot - Poles vs Conversion Rate
plt.figure(figsize=(10, 6))
sns.scatterplot(
    data=pole_conversion,
    x='Poles',
    y='ConversionRate',
    hue='PoleDriver',
    s=120
)
for _, row in pole_conversion.iterrows():
    plt.text(row['Poles'] + 0.2, row['ConversionRate'], row['PoleDriver'], fontsize=9)

plt.xlabel("Number of Poles")
plt.ylabel("Pole → Win Conversion Rate (%)")
plt.title("F1 2021–2025: Poles vs Conversion Rate (≥3 Poles)")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.savefig("../graphs/poles_vs_conversion.png")
plt.show()

display(Markdown("""
**Graph 1: F1 2021-2025: Poles vs Conversion Rate (≥3 Poles)**  
Scatter plot showing the relationship between pole positions and win conversion rate.  
- X-axis = total number of pole positions  
- Y-axis = percentage of poles converted to wins  
- Each point = a driver with at least 3 poles  
- Driver labels appear next to each point
"""))

display(Markdown("""
**Poles vs Conversion Rate**  
Verstappen dominates with 50 poles and a remarkable 77% conversion rate, far exceeding all competitors. Piastri shows strong efficiency with 68% conversion from just 6 poles. Russell (57%) and Hamilton (50%) demonstrate solid conversion despite fewer poles (8 and 4 respectively). Leclerc's low 17% conversion from 18 poles is most likely from Ferarri's strategy which has cost him a lot of races. while Norris at 50% from 18 poles shows average success compared to Lecrec. Perez and Bottas struggle with conversion rates below 35%.
"""))


# Graph 2: Lap 1 position change
plt.figure(figsize=(12, 6))
sns.countplot(x='PoleDriver', hue='PoleStartVsLap1', data=df_filtered)
plt.ylabel("Number of Races")
plt.title("Lap 1 Position Change for Pole Sitters (≥3 Poles)")
plt.tight_layout()
plt.savefig("../graphs/pole_start_vs_lap1.png")
plt.show()

display(Markdown("""
**Graph 2: Lap 1 Position Change for Pole Sitters (≥3 Poles)**  
Stacked bar chart showing what happens to pole position by the end of Lap 1.  
- Blue bars = gained positions (moved up from pole)  
- Orange bars = maintained pole position  
- Green bars = lost positions (dropped back)  
- Height shows total pole starts for each driver
"""))

display(Markdown("""
**Lap 1 Position Change for Pole Sitters**  
Verstappen demonstrates exceptional Lap 1 consistency, maintaining pole position in approximately 34 of 48 starts, with minimal position losses. Leclerc shows balanced performance across all three categories with roughly equal distribution. Norris maintains pole in about 10 of 17 starts but shows notable position losses. Piastri, Hamilton, and Russell exhibit strong maintenance rates relative to their total poles. Perez and Bottas show more vulnerability on Lap 1, with relatively higher loss frequencies compared to maintained positions.
"""))


# Graph 3: Track PCI
track_pci = df_all.groupby('Circuit')['WonFromPole'].mean().reset_index()
track_pci['PCI'] = (track_pci['WonFromPole'] * 100).round(1)
track_pci = track_pci.sort_values('PCI', ascending=False)

plt.figure(figsize=(14, 6))
sns.barplot(x='Circuit', y='PCI', data=track_pci)
plt.xticks(rotation=45, ha='right')
plt.ylabel("Pole → Win Conversion Rate (%)")
plt.title("Track Pole Conversion Index (2021–2025)")
plt.tight_layout()
plt.savefig("../graphs/track_pci.png")
plt.show()

display(Markdown("""
**Graph 3: Track Pole Conversion Index (2021-2025)**  
Bar chart showing pole-to-win conversion rate for each circuit.  
- X-axis = circuit name  
- Y-axis = percentage of races where pole position won  
- Higher bars = tracks where starting P1 is more advantageous  
- Sorted from highest to lowest conversion rate
"""))

display(Markdown("""
**Track Pole Conversion Index**  
Seven circuits show perfect 100% pole conversion: Abu Dhabi, Dutch, Chinese, Styrian Park, Turkish, Japanese, and Saudi Arabian GPs, indicating pole position is nearly insurmountable at these venues. Bahrain (80%), Qatar (75%), and Singapore/Australian/Canadian GPs (around 75%) also heavily favor pole starters. Mid-range circuits like Monaco (60%), São Paulo (60%), and Spanish (60%) offer moderate overtaking opportunities. The lowest conversion rates appear at Azerbaijan (20%), Italian (20%), and Hungarian (20%) GPs, plus several circuits at 0%, suggesting these tracks enable strategic overtakes or feature high incident rates that disrupt the pole advantage.
"""))


# Graph 4: Pole to finish change
plt.figure(figsize=(12, 6))
sns.boxplot(x='PoleDriver', y='PoleToFinishChange', data=df_filtered)
plt.axhline(0, linestyle='--', color='gray')
plt.ylabel("Pole → Finish Change (1 - FinishPos)")
plt.title("Pole to Finish Position Change (≥3 Poles)")
plt.tight_layout()
plt.savefig("../graphs/pole_to_finish_change.png")
plt.show()

display(Markdown("""
**Graph 4: Pole to Finish Position Change (≥3 Poles)**  
Boxplot showing how drivers' finishing positions compare to their pole starts.  
- Y-axis = position change (1 - finish position; 0 = won from pole, negative = lost positions)  
- Gray dashed line at 0 = winning from pole  
- Box = interquartile range (middle 50% of results)  
- Whiskers = typical range; circles = outlier races
"""))

display(Markdown("""
**Pole to Finish Position Change**  
Verstappen and Piastri show the tightest distributions centered near 0 (winning from pole), with Verstappen's median slightly above zero and minimal negative outliers, demonstrating exceptional race execution. Hamilton and Perez maintain compact distributions around -1 to -2 (podium finishes). Bottas shows the widest spread with extreme negative outliers reaching -13 to -18, indicating inconsistent race performances. Leclerc, Norris, Sainz, and Russell display moderate spreads with medians around -2 to -3, suggesting they typically finish on the podium but rarely convert poles to wins. The horizontal gray line represents race wins from pole. Drivers with boxes closer to this line have better pole conversion consistency.
"""))


# Lap 1 stats
lap1_stats = df_filtered.groupby('PoleDriver').agg(
    Poles=('PoleFinishPosition', 'count'),
    AvgLap1Change=('Lap1Change', 'mean')
).reset_index()

lap1_stats['AvgLap1Change'] = lap1_stats['AvgLap1Change'].round(1)
lap1_stats = lap1_stats.sort_values('AvgLap1Change')

# Save outputs
df_filtered.to_csv('poles_2021_2025_3plus_full.csv', index=False)
pole_conversion.to_csv('pole_conversion_driver_2021_2025_3plus.csv', index=False)
lap1_stats.to_csv('lap1_stats_driver_2021_2025_3plus.csv', index=False)
track_pci.to_csv('track_pci_2021_2025.csv', index=False)
